In [1]:
# ---------- IMPORT MODULES ----------
from get_city_graph import get_city_graph
from get_city_data_from_db import get_city_data_from_db
from store_city_to_db import store_city_to_db
from get_or_create_run_config import get_or_create_run_config
from create_iteration import create_iteration
from generate_vehicles import generate_vehicles
from generate_vehicle_routes import generate_vehicle_routes
from generate_congestion import generate_congestion
from plot_congestion_heatmap import plot_congestion_heatmap, plot_congestion_heatmap_interactive
from filter_routes_for_qubo import filter_routes_for_qubo
from get_congestion_weights import get_congestion_weights
from normalize_congestion_weights import normalize_congestion_weights
from congestion_weights import congestion_weights
from qubo_matrix import qubo_matrix

# ---------- CONFIGURATION ----------
from sqlalchemy.orm import sessionmaker
from models import * #City, Node, Edge, RunConfig, Iteration, Vehicle, VehicleRoute, CongestionMap, RoutePoint  # adjust to your actual model imports


CITY_NAME = "Bratislava, Slovakia"#"Most pri Bratislave, Slovakia"
DIST_THRESH = 10
SPEED_DIFF_THRESH = 2
RUN_CONFIG_ID = 3#20      
ITERATION_ID = 4#1
API_KEY = ''

Session = sessionmaker(bind=engine)
session = Session()


city = session.query(City).filter_by(name=CITY_NAME).first()
_, edges = get_city_data_from_db(session, city.id)

Connection to MariaDB successful!


In [2]:
def get_run_config_params(session, run_config_id, city_name):
    from sqlalchemy import text

    # Step 1: Get city_id
    city_id_result = session.execute(text("""
        SELECT id FROM cities WHERE name = :city_name
    """), {'city_name': city_name}).fetchone()

    if city_id_result is None:
        raise ValueError(f"No city found with name '{city_name}'")

    city_id = city_id_result[0]

    sql = text("""
        SELECT
            id,
            city_id,
            n_cars,
            k_alternatives,
            min_length,
            max_length,
            time_step,
            time_window,
            created_at
        FROM run_configs
        WHERE id = :run_config_id
    """)

    result = session.execute(sql, {'run_config_id': run_config_id, 'city_id': city_id}).fetchone()

    if result is None:
        raise ValueError(f"No run_config found with id={run_config_id}")

    # Return as dictionary
    keys = ['id', 'city_id', 'n_cars', 'k_alternatives', 'min_length',
            'max_length', 'time_step', 'time_window', 'created_at']
    return dict(zip(keys, result))


In [3]:
config = get_run_config_params(session, run_config_id=RUN_CONFIG_ID, city_name = CITY_NAME)
print(config['n_cars'], config['k_alternatives'], config['min_length'])

10000 3 200


In [4]:
def get_routes_df(session, run_config_id, iteration_id):
    from sqlalchemy import text
    import pandas as pd

    query = text("""
        SELECT
            vehicle_id,
            route_id,
            point_id,
            edge_id,
            lat,
            lon,
            time,
            speed,
            cardinal
        FROM trafficOptimization.route_points
        WHERE run_configs_id = :run_config_id
          AND iteration_id = :iteration_id
        ORDER BY vehicle_id, route_id, point_id
    """)

    result = session.execute(query, {
        'run_config_id': run_config_id,
        'iteration_id': iteration_id
    })

    return pd.DataFrame(result.fetchall(), columns=[
        'vehicle_id', 'route_id', 'point_id', 'edge_id',
        'lat', 'lon', 'time', 'speed', 'cardinal'
    ])


In [5]:
def get_congestion_df(session, run_config_id, iteration_id):
    from sqlalchemy import text
    import pandas as pd

    query = text("""
        SELECT
            edge_id,
            vehicle1,
            vehicle2,   
            vehicle1_route,
            vehicle2_route,
            congestion_score
        FROM trafficOptimization.congestion_map
        WHERE run_configs_id = :run_config_id
          AND iteration_id = :iteration_id
    """)

    result = session.execute(query, {
        'run_config_id': run_config_id,
        'iteration_id': iteration_id
    })

    return pd.DataFrame(result.fetchall(), columns=['edge_id', 'vehicle1', 'vehicle2', 'vehicle1_route', 'vehicle2_route', 'congestion_score'])


In [6]:
session

In [7]:
#routes_df = get_routes_df(session, RUN_CONFIG_ID, ITERATION_ID)
congestion_df = get_congestion_df(session, RUN_CONFIG_ID, ITERATION_ID)


# Step 8: Filter routes for QUBO
filtered_vehicles = filter_routes_for_qubo(congestion_df, threshold_percentile=0.9)
#print(filtered_vehicles)
N_FILTERED = len(filtered_vehicles)
print("Number of elements:", N_FILTERED)

# Step 9: Compute wights from congestion
weights_df = get_congestion_weights(session, RUN_CONFIG_ID, ITERATION_ID)
#print(weights_df)
weights_df.to_csv("files/weights_df.csv", index=False)

#weights_normalized = normalize_congestion_weights(weights_df, N_FILTERED, config['k_alternatives'], filtered_vehicles)
#weights_wo_normalization, max_weight = congestion_weights(weights_df, N_FILTERED, config['k_alternatives'], filtered_vehicles)
#print(weights_normalized)



Threshold (90.00%): 16.196978
Filtered vehicles (549 total): [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 111, 113, 115, 118, 119, 121, 122, 123, 124, 126, 128, 129, 130, 132, 133, 134, 136, 137, 145, 146, 147, 150, 151, 152, 154, 157, 158, 159, 160, 161, 167, 169, 170, 171, 172, 173, 175, 176, 177, 184, 185, 186, 187, 189, 191, 194, 197, 200, 203, 204, 206, 208, 209, 211, 216, 217, 219, 220, 224, 230, 233, 238, 246, 249, 253, 254, 256, 257, 259, 268, 270, 272, 282, 285, 286, 290, 295, 296, 298, 299, 308, 309, 310, 312, 317, 318, 319, 321, 324, 326, 327, 329, 336, 344, 346, 349, 353, 358, 362, 363, 364, 365, 368, 370, 372, 373, 374, 380, 383, 385, 387, 394, 3

In [8]:
# Step 10: QUBO
Q, weights, w_c = qubo_matrix(N_FILTERED, config['k_alternatives'], weights_df, filtered_vehicles, lambda_strategy="normalized", fixed_lambda=1.0)

#for (q1, q2), value in Q.items():
#    print(f"Q[{q1}, {q2}] = {value:.4f}")


Vehicle ids: [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 111, 113, 115, 118, 119, 121, 122, 123, 124, 126, 128, 129, 130, 132, 133, 134, 136, 137, 145, 146, 147, 150, 151, 152, 154, 157, 158, 159, 160, 161, 167, 169, 170, 171, 172, 173, 175, 176, 177, 184, 185, 186, 187, 189, 191, 194, 197, 200, 203, 204, 206, 208, 209, 211, 216, 217, 219, 220, 224, 230, 233, 238, 246, 249, 253, 254, 256, 257, 259, 268, 270, 272, 282, 285, 286, 290, 295, 296, 298, 299, 308, 309, 310, 312, 317, 318, 319, 321, 324, 326, 327, 329, 336, 344, 346, 349, 353, 358, 362, 363, 364, 365, 368, 370, 372, 373, 374, 380, 383, 385, 387, 394, 396, 398, 407, 413, 417, 422, 425, 434, 437, 448,

In [10]:
import numpy as np
import pandas as pd

def qubo_dict_to_dataframe(Q, size):
    matrix = np.zeros((size, size))
    for (i, j), v in Q.items():
        matrix[i][j] = v
        if i != j:
            matrix[j][i] = v  # ensure symmetry for display
    return pd.DataFrame(matrix), matrix

# Example usage
size = N_FILTERED * config['k_alternatives']
Q_df, matrix = qubo_dict_to_dataframe(Q, size)



In [11]:
Q_df.to_csv("files/qubo_matrix.csv", index=False)


In [12]:
from compute_shortest_routes import compute_shortest_routes

# Step : Plot heatmap
plot_map = plot_congestion_heatmap_interactive(edges, congestion_df,offset_deg=0.000025)
plot_map
plot_map.save("files/congestion_heatmap.html")

shortes_routes_dur_df = compute_shortest_routes(session, RUN_CONFIG_ID, ITERATION_ID, method="duration")
plot_map_dur = plot_congestion_heatmap_interactive(edges, shortes_routes_dur_df,offset_deg=0.000025)
plot_map_dur
plot_map_dur.save("files/shortest_routes_dur_congestion_heatmap.html")


shortes_routes_dis_df = compute_shortest_routes(session, RUN_CONFIG_ID, ITERATION_ID, method="distance")
plot_map_dis = plot_congestion_heatmap_interactive(edges, shortes_routes_dis_df,offset_deg=0.000025)
plot_map_dis
plot_map_dis.save("files/shortest_routes_dis_congestion_heatmap.html")

session.close()